In [ ]:
# DATASET INSPECTION
import os
import pandas as pd

# Define metadata path and image directories
metadata_path = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_metadata.csv'
image_dir_1 = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_1'
image_dir_2 = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_2'

# Load metadata
metadata = pd.read_csv(metadata_path)

# Inspect the first few rows
print("##### Inspect The First Few Rows #####")
print(metadata.head())
print("\n")

# Check data types
data_types = metadata.dtypes
print("##### Data Types ##### \n", data_types)
print("\n")

# Check for missing values
missing_values = metadata.isnull().sum()
print("##### Missing Values In Columns ##### \n", missing_values)
print("\n")

# Check Unknown Valuses In "SEX" and "LOCALIZATION" Columns
print("##### Checking 'unknown' values in 'SEX' and 'LOCALIZATION' columns #####")
if 'unknown' in metadata['sex'].values:
    print("'unknown' exists in 'sex' column.")
else:
    print("'unknown' does not exist in 'sex' column.")

if 'unknown' in metadata['localization'].values:
    print("'unknown' exists in 'localization' column.")
else:
    print("'unknown' does not exist in 'localization' column.")

# Check for duplicate rows in metadata
print("\n")
print("##### Checking Duplicate Rows #####")
duplicate_count = metadata.duplicated().sum()
if duplicate_count > 0:
    print(f"Duplicate rows found: {duplicate_count}. Removing duplicates...")
    # metadata.drop_duplicates(inplace=True)
    # print("Duplicate rows removed.")
else:
    print("No duplicate rows found. No action taken.")

# Verify that all images listed in the metadata are present in the folders
print("\n")
print("##### Checking Missing Images In Folders #####")
image_ids = set(metadata['image_id'])
all_image_files = set(os.listdir(image_dir_1) + os.listdir(image_dir_2))
missing_images = [img_id for img_id in image_ids if f"{img_id}.jpg" not in all_image_files]
if missing_images:
    print(f"Missing images: {missing_images}")
else:
    print("All images are accounted for.")

# Check for duplicate image_id values
print("\n")
print("##### Checking Duplicate Image_id's #####")
duplicate_image_ids = metadata[metadata['image_id'].duplicated()]['image_id']
if not duplicate_image_ids.empty:
    print(f"Duplicate image IDs found: {duplicate_image_ids.tolist()}")
else:
    print("No duplicate image IDs found.")

# Basic statistics of the dataset
print("\n")
print("##### Statistics of the dataset #####")
print(metadata.describe(include='all'))
print("\n")


In [ ]:
# DATASET VISUALIZATION

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

metadata_path = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_metadata.csv'
image_dir_1 = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_1'
image_dir_2 = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_images_part_2'

# Load metadata
metadata = pd.read_csv(metadata_path)

# Visualize the distribution of diseases
print("\n")
print("##### Displaying Distribution of Diseases (Lesion) #####")
sns.countplot(data=metadata, x='dx')
plt.title("Distribution of Skin Diseases in HAM10000 Dataset")
plt.xticks(rotation=45)
plt.show()

# Display sample images
print("##### Displaying sample images #####")
def display_sample_images(image_dir, num_samples=5):
    sample_files = os.listdir(image_dir)[:num_samples]
    plt.figure(figsize=(10, 5))
    for i, filename in enumerate(sample_files):
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path)
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.show()

display_sample_images(image_dir_1)
display_sample_images(image_dir_2)
print("\n")

In [ ]:
# DATASET MANIPULATION : HANDLING UNKNOW VALUES IN "SEX" AND "LOCALIZATION" COLUMN'S

import pandas as pd
# Define metadata path and image directories
metadata_path = r'D:\skin_disease_detection\backend\data\Ham10000\HAM10000_metadata.csv'
# Load metadata
metadata = pd.read_csv(metadata_path)

# HANDLING UNKNOW VALUES IN "SEX" AND "LOCALIZATION" COLUMN'S
Handle missing values for 'sex' and 'localization' columns
if 'unknown' in metadata['sex'].values:
    metadata['sex'].replace('unknown', metadata['sex'].mode()[0], inplace=True)
    print(f"'unknown' in 'sex' replaced with mode: {metadata['sex'].mode()[0]}")

if 'unknown' in metadata['localization'].values:
    metadata['localization'].replace('unknown', metadata['localization'].mode()[0], inplace=True)
    print(f"'unknown' in 'localization' replaced with mode: {metadata['localization'].mode()[0]}")

# Save the cleaned metadata
metadata.to_csv(metadata_path, index=False)
print("Cleaned metadata saved to metadata.csv : {metadata_path}")
print("\n")